## Analítica de Tickets OTRS CI Antecedente - Servicio Consecuente

In [1]:
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt 
import mysql.connector
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules

#### Conexión directa al MySQL del OTRS

In [2]:
otrs_user = 'usr_tronexotrs'
otrs_pass = 'usr$otrs#56aN7Rt*mp1'

In [3]:
cnx = mysql.connector.connect(user=otrs_user, password=otrs_pass,
                              host='pantermysql.cwqmr5tsjjh9.us-west-2.rds.amazonaws.com',
                              database='tronex.otrs')

In [6]:
query = ('SELECT T.title, S.* FROM `tronex.otrs`.itv_sabana_cmdb AS S INNER JOIN `tronex.otrs`.ticket AS T \
         ON S.ticket_id = T.id WHERE queue_id = 6 AND \
         customer_id NOT IN ("soporte@sasaconsultoria.com", "telefonia@tronex.com"\
         , "serviciosit@tronex.com") AND T.title not like "%has low free space" \
        AND user_id in (39, 53, 5) \
        AND ticket_state_id <> 9')

df = pd.read_sql(query, con=cnx)

## Datamining - Reglas de asociación

In [9]:
df.head()

,title,ticket_id,ticket_numero,elemento_cmdb,servicio,tipo_ticket,fecha_creacion,fecha,anio,mes,dia,fecha_generacion_consulta
0,Error crear cliente ax,57971,2020050206000064,AX,ERP::ADMINISTRACION,REQUERIMENTO/SOLICITUD,2020-05-02 09:35:04,2020-05-02,2020,5,2,2021-06-17 07:42:11
1,Tiempo AX,57979,2020050406000015,AX,ERP::ADMINISTRACION,REQUERIMENTO/SOLICITUD,2020-05-04 06:20:04,2020-05-04,2020,5,4,2021-06-17 07:42:11
2,Novedad impresora,57980,2020050406000024,BOGT_P1_BN_02,IMPRESION ::IMPRESION,REQUERIMENTO/SOLICITUD,2020-05-04 06:50:04,2020-05-04,2020,5,4,2021-06-17 07:42:11
3,Subir plantillas AX,57985,2020050406000079,AX,ERP::ADMINISTRACION,REQUERIMENTO/SOLICITUD,2020-05-04 08:20:06,2020-05-04,2020,5,4,2021-06-17 07:42:11
4,Falla fortyclient,57986,2020050406000088,FORTICLIENTE - 5.4.1.0840,OFIMATICA::APLICACIONES DE ESCRITORIO,REQUERIMENTO/SOLICITUD,2020-05-04 09:05:04,2020-05-04,2020,5,4,2021-06-17 07:42:11


In [12]:
df['cantidad_servicio'] = 1

df_pivot = df.pivot_table(index=['fecha_creacion', 'elemento_cmdb'], \
                          columns='servicio', \
                          aggfunc='sum', \
                          values='cantidad_servicio', \
                          fill_value=0)

def encode(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

df_pivot = df_pivot.applymap(encode)

In [13]:
df_pivot.head()

,servicio,ACCESIBILIDAD::DIRECTORIO ACTIVO,AUTOMATIZACION DE LA ADMINISTRACION TI::BACKUP,AUTOMATIZACION DE LA ADMINISTRACION TI::CABLEADO ESTRUCTURADO,AUTOMATIZACION DE LA ADMINISTRACION TI::GESTION TIC,AUTOMATIZACION DE LA ADMINISTRACION TI::SEGURIDAD DE LA INFORMACIÓN,AUTOMATIZACION DE LA ADMINISTRACION TI::SERVICIOS EN LA NUBE,AUTOMATIZACION DE POST VENTA Y SOPORTE,AUTOMATIZACION FUERZA DE VENTAS::ADMINISTRACION,AUTOMATIZACION FUERZA DE VENTAS::CIERRE DE DIA,AUTOMATIZACION FUERZA DE VENTAS::INICIO DE DIA,...,NAVEGACION::INALAMBRICA::WIFI,NAVEGACION::POR CABLE,OFIMATICA,OFIMATICA::APLICACIONES DE ESCRITORIO,OFIMATICA::EQUIPOS DE COMPUTO,OFIMATICA::PERIFERICO,OFIMATICA::SOLICITUD DE RECURSO,TELEFONIA::TELEFONIA CONMUTADA,TELEFONIA::TELEFONIA IP,TELEFONIA::TELEFONIA MOVIL
fecha_creacion,elemento_cmdb,,,,,,,,,,,,,,,,,,,,,
2020-05-02 09:35:04,AX,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-05-04 06:20:04,AX,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-05-04 06:50:04,BOGT_P1_BN_02,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-05-04 08:20:06,AX,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-05-04 09:05:04,FORTICLIENTE - 5.4.1.0840,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


### Cálculo de las reglas de asociación
---

* Se calculan las reglas de asociación usando una presencia de los datos en un 30% de las muestras `(soporte)`

* El tamaño de la muestra es un día `(no es cada ticket)`

In [ ]:
df_freqserv = apriori(df_pivot, min_support=0.3, use_colnames=True)
df_rules = association_rules(df_freqserv, min_threshold=1, metric='lift')

### Cálculo de las reglas de asociación
#### Explicación de variables
---
* El antecedente implica el consecuente, no es conmutativo
* Support: Es la presencia en el dataset del antecedente, el consecuente y la regla en si
* Confidence: Es el porcentaje de los casos en el que la regla es cierta

* Lift: Cuando el valor es 1 son variables independientes, si es mayor son dependientes y si es menor son inversamente dependientes
* Leverage: Es similar al lift pero se calcula diferente y el valor de independencia es 0
* Conviction: es el error que habría en caso de que las variables sean independientes

### Cálculo de las reglas de asociación
---

In [ ]:
df_rules.loc[(df_rules.confidence>0.5)].sort_values('confidence', \
                                                    ascending=False).head(10)

In [ ]:
df_rules['antecedents'] = df_rules['antecedents'].apply(lambda x: set(x))
df_rules['consequents'] = df_rules['consequents'].apply(lambda x: set(x))

In [ ]:
df_rules[df_rules['antecedents'] == {'TELEFONIA::TELEFONIA IP'}]

In [ ]:
df['fecha_creacion'].nunique()

In [ ]:
fechas = df[df['servicio'] == 'TELEFONIA::TELEFONIA IP']['fecha_creacion'].unique()

In [ ]:
fechas = df.loc[(df['fecha_creacion'].isin(fechas)) &\
                (df['servicio'] == 'MENSAJERIA Y COLABORACION::SERVIDOR DE ARCHIVOS')]['fecha_creacion'].unique()

In [ ]:
df_temp = df.loc[(df['fecha_creacion']
        .isin(fechas)) & (df['servicio']
                          .isin(['TELEFONIA::TELEFONIA IP']))]

In [ ]:
df_temp['fecha_creacion'] = pd.to_datetime(df_temp['fecha_creacion'])

In [ ]:
df_ant = df_temp.set_index('fecha_creacion').groupby([pd.Grouper(freq='D'), 'servicio'])\
        .agg({'title':'count'}).reset_index('servicio')

In [ ]:
df_temp = df.loc[(df['fecha_creacion']
        .isin(fechas)) & (df['servicio']
                          .isin(['MENSAJERIA Y COLABORACION::SERVIDOR DE ARCHIVOS']))]

In [ ]:
df_temp['fecha_creacion'] = pd.to_datetime(df_temp['fecha_creacion'])

In [ ]:
df_con = df_temp.set_index('fecha_creacion')\
        .groupby([pd.Grouper(freq='D'), 'servicio']).agg({'title':'count'}).reset_index('servicio')

In [ ]:
df_con.sort_values(by='title', ascending=False)

In [ ]:
df_con = df_con.loc[df_con['title'] < 64]

In [ ]:
plt.ion()
plt.figure(figsize=(16,8))
plt.subplot(211)
plt.plot(df_ant.index, df_ant.title, label='ant', color='red')
plt.subplot(212)
plt.plot(df_con.index, df_con.title, label='con', color='blue')
plt.legend()
plt.show()

In [ ]:
df.loc[(df['fecha_creacion']
        .isin(fechas)) & (df['servicio']
                          .isin(['TELEFONIA::TELEFONIA IP', 'MENSAJERIA Y COLABORACION::SERVIDOR DE ARCHIVOS']))]

In [ ]:
df['agente'].unique()

In [ ]:
df[df['servicio'] == 'ERP::ADMINISTRACION']['agente'].unique()